In [1]:
import curses
from curses import KEY_RIGHT, KEY_LEFT, KEY_UP, KEY_DOWN
from random import randint
import tensorflow as tf
from tensorflow import layers
import numpy as np
import os
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression
import tflearn

# tf.enable_eager_execution()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'


In [2]:
def training_model():
    network = input_data(shape=[None, 2, 1], name='input')
    network = fully_connected(network, 1, activation='linear')
    network = regression(network, optimizer='adam', learning_rate=1e-2, loss='mean_square', name='target')
    model = tflearn.DNN(network, tensorboard_dir='log')
    return model



In [3]:
training_data = []

In [4]:
training_data.append([np.array([1, 256]), np.array(1)])
training_data.append([np.array([1, 256]), np.array(1)])
training_data.append([np.array([0, 256]), np.array(0)])
training_data.append([np.array([0, 256]), np.array(0)])
training_data.append([np.array([0, 256]), np.array(0)])

In [5]:
list((i[1] for i in training_data))

[array(1), array(1), array(0), array(0), array(0)]

In [6]:
x = np.array([i[0] for i in training_data]).reshape(-1, 2, 1)
y = np.array([i[1] for i in training_data]).reshape(-1, 1)
model = training_model()
model.fit(x, y, n_epoch=10, shuffle=True)

---------------------------------
Run id: FP8MGI
Log directory: log/
---------------------------------
Training samples: 5
Validation samples: 0
--
Training Step: 1  | time: 0.049s
| Adam | epoch: 001 | loss: 0.00000 -- iter: 5/5
--
Training Step: 2  | total loss: 34.35497 | time: 0.003s
| Adam | epoch: 002 | loss: 34.35497 -- iter: 5/5
--
Training Step: 3  | total loss: 16.95657 | time: 0.003s
| Adam | epoch: 003 | loss: 16.95657 -- iter: 5/5
--
Training Step: 4  | total loss: 5.17932 | time: 0.003s
| Adam | epoch: 004 | loss: 5.17932 -- iter: 5/5
--
Training Step: 5  | total loss: 3.44247 | time: 0.002s
| Adam | epoch: 005 | loss: 3.44247 -- iter: 5/5
--
Training Step: 6  | total loss: 5.85746 | time: 0.008s
| Adam | epoch: 006 | loss: 5.85746 -- iter: 5/5
--
Training Step: 7  | total loss: 7.27341 | time: 0.003s
| Adam | epoch: 007 | loss: 7.27341 -- iter: 5/5
--
Training Step: 8  | total loss: 6.61515 | time: 0.003s
| Adam | epoch: 008 | loss: 6.61515 -- iter: 5/5
--
Training Step:

In [7]:
model.predict(np.array([0, 256]).reshape(-1, 2, 1))

array([[-1.2534963]], dtype=float32)